# RMSD visualization

This notebooks aims to identify a set of docking poses that can be used to visualize different RMSD ranges.

In [1]:
from pathlib import Path

from openeye import oechem
import pandas as pd
from pymol import cmd

from kinoml.databases.pdb import smiles_from_pdb, download_pdb_structure
from kinoml.modeling.OEModeling import read_molecules, select_chain, remove_non_protein, write_molecules

In [2]:
CACHE_DIR = Path("../data/.cache")

In [3]:
fred_results = pd.read_csv("../data/fred_results.csv", index_col=0)
fred_results = fred_results[~fred_results["rmsd"].isna()]
fred_results = fred_results.sort_values("rmsd")

## Select example

In [4]:
good_poses_pdb_ids = set(fred_results[
    fred_results["rmsd"] < 2
]["ligand_pdb_id"])
worst_poses_pdb_ids = set(fred_results[
    fred_results["rmsd"] > 30
]["ligand_pdb_id"])
print("Structures with good and bad poses:")
print([x for x in good_poses_pdb_ids if x in worst_poses_pdb_ids])

Structures with good and bad poses:
['3myg', '5zan', '4qmp', '2x6d', '4qmt', '2x6e', '4qmv']


In [5]:
print(fred_results[fred_results["ligand_pdb_id"] == "5zan"]["rmsd"].to_list())

[1.2000502203033006, 1.2538422697851594, 1.2764521877062218, 1.448073267310739, 1.4820856398164042, 1.509142813404351, 1.532188834641475, 1.6786712565299973, 1.7489310127903848, 1.820807985140113, 1.8288673919942915, 1.9032821486842144, 1.9488099814373891, 1.9829243713515647, 1.9875244809813035, 1.996838383858343, 2.1799453261492587, 2.226179645154452, 2.772638420800664, 3.300165975939695, 3.323934058461449, 3.405509814073364, 3.4351824414796366, 3.4628937456627225, 3.502713827341593, 3.508501785841073, 3.51044046381647, 3.5628226476699623, 3.5835129935581373, 3.5993398526674296, 3.6014824378302883, 3.629737314014611, 3.6365351684329967, 3.640497772455025, 3.640976487324519, 3.65067423567209, 3.651334704597759, 3.6695214946911, 3.686275038063492, 3.729477606683005, 3.743648462729908, 3.824340559985473, 3.84495460798954, 3.845087202359655, 3.860446067975824, 3.871290736789217, 3.8874304455192497, 4.006192877907902, 4.2321717422914205, 4.242827624356192, 4.278237078517272, 4.291001456536

In [6]:
# find representative docking poses for several RMSD ranges
pdb_id, expo_id  = "5zan", "9A6"
result_selection = fred_results[fred_results["ligand_pdb_id"] == pdb_id]
representative_poses = []
rmsds = []
for lower_bound in [0, 2, 4, 8, 16, 30]:
    range_poses = result_selection[
        (result_selection["rmsd"] >= lower_bound) &
        (result_selection["rmsd"] < lower_bound + 2) 
    ]
    if len(range_poses) > 0:
        rmsds.append(range_poses.rmsd[0])
        representative_poses.append(range_poses.index[0])
print(representative_poses)
print(rmsds)

['kinoml_OEFredDockingFeaturizer_Human_AurA_rcsb_3nrm_chainA_altlocNone_5zan-9A6_ligand.sdf_4', 'kinoml_OEFredDockingFeaturizer_Human_AurA_rcsb_5one_chainA_altlocNone_5zan-9A6_ligand.sdf_1', 'kinoml_OEFredDockingFeaturizer_Human_AurA_rcsb_5ew9_chainA_altlocNone_5zan-9A6_ligand.sdf_2', 'kinoml_OEFredDockingFeaturizer_Human_AurA_rcsb_4ceg_chainA_altlocNone_5zan-9A6_ligand.sdf_4', 'kinoml_OEFredDockingFeaturizer_Human_AurA_rcsb_4dea_chainA_altlocNone_5zan-9A6_ligand.sdf_5', 'kinoml_OEFredDockingFeaturizer_Human_AurA_rcsb_4c3p_chainA_altlocNone_5zan-9A6_ligand.sdf_4']
[1.2000502203033006, 2.1799453261492587, 4.006192877907902, 8.24113709159725, 16.031509016426057, 30.13180490206735]


In [7]:
# merge pose with protein and save
docking_pose_paths = list(Path("../data/fred").glob("*.sdf"))
for representative_pose in representative_poses:
    protein_pdb_id = representative_pose.split("_")[5]
    protein_chain_id = representative_pose.split("_")[6][5]
    protein_structure_path = download_pdb_structure(protein_pdb_id, CACHE_DIR)
    protein_structure = read_molecules(protein_structure_path)[0]
    protein_structure = select_chain(protein_structure, protein_chain_id)
    protein_structure = remove_non_protein(protein_structure, remove_water=True)
    docking_pose_path = [
        docking_pose_path for docking_pose_path 
        in docking_pose_paths 
        if protein_pdb_id in docking_pose_path.name and "5zan" in docking_pose_path.name
    ][0]
    docking_pose = read_molecules(docking_pose_path)[0]
    oechem.OEAddMols(protein_structure, docking_pose)
    write_molecules([protein_structure], f"../data/fred/{protein_pdb_id}_{expo_id}.pdb")

## PyMol visualization

In [8]:
# load reference structure, docking pose and align
cmd.fetch(pdb_id)
for representative_pose in representative_poses:
    protein_pdb_id = representative_pose.split("_")[5]
    cmd.load(f"../data/fred/{protein_pdb_id}_{expo_id}.pdb", f"{protein_pdb_id}")
    cmd.align(f"{protein_pdb_id}", pdb_id)

 ExecutiveLoad-Detail: Detected mmCIF


In [9]:
# representation of reference structure
cmd.show_as("cartoon", f"{pdb_id}")
cmd.show_as("licorice", f"{pdb_id} and resname {expo_id}")
cmd.color("white", pdb_id)

In [10]:
# color
color_dict = {
    "3nrm": "green", "5one": "cyan", "5ew9": "magenta", "4ceg": "salmon", 
    "4dea": "slate", "4c3p": "orange",
}
for docking_pose, color in color_dict.items():
    cmd.color(color, docking_pose)
cmd.color("atomic", "not element C")

In [11]:
# docking pose representation
for docking_pose in color_dict.keys():
    cmd.hide("cartoon", docking_pose)
    cmd.hide("licorice", docking_pose)
    cmd.show_as("lines", f"{docking_pose} and resname UNL")
cmd.set("line_width", 3)
cmd.hide("lines", "hydro")

In [12]:
# general visualization settings
cmd.bg_colour("white")
cmd.remove("solvent")
cmd.set("ambient", 0.4)
cmd.set("antialias", 1)
cmd.set("ortho", 1)
cmd.set("ray_trace_mode", 1)
cmd.set("cartoon_fancy_helices", 1)
cmd.set("cartoon_highlight_color", "grey30")

In [13]:
# save pymol session
cmd.save("../pics/rmsd_visualization.pse")

In [14]:
# pic for binding pocket focus
cmd.set_view((
    -0.641249239,    0.734733999,   -0.221281186,
    -0.678720772,   -0.677629352,   -0.283111989,
    -0.357958853,   -0.031358212,    0.933209717,
     0.000077804,    0.000069283, -165.494964600,
   -14.712101936,   31.087493896,   -7.455860615,
   141.041702271,  189.951400757,   20.000000000
))
cmd.ray(4000, 4000)
cmd.png("../pics/rmsd_visualization.png")

1

In [15]:
# pic for ligand focus
cmd.set("cartoon_transparency", 0.8)
cmd.set_view((
    -0.653953254,   -0.321630001,   -0.684761167,
    -0.669361591,   -0.175815970,    0.721829236,
    -0.352553338,    0.930398583,   -0.100310877,
     0.000077804,    0.000069283,  -79.589012146,
   -14.712101936,   31.087493896,   -7.455860615,
    55.135738373,  104.045455933,   20.000000000
))
cmd.ray(4000, 2000)
cmd.png("../pics/rmsd_visualization2.png")

1